Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
# filePath = "dbfs:/FileStore/tables/Files/names.csv"
# namesDf = spark.read.format("csv")
#               .option("header","true")
#               .option("inferSchema","true")
#               .load(filePath)

# display(namesDf)
from pyspark.sql.functions import col, unix_timestamp, lit, round, year, when, split, avg
from datetime import datetime
filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

display(namesDf.limit(20))
namesDf.explain()
namesDf.columns
namesDf = namesDf.withColumn("CurrTime", lit(unix_timestamp(lit(datetime.now()))))

namesDf = namesDf.withColumn("Feet", round(col("height") * 0.032, 2))

tempDf = namesDf.withColumn("FirstName", split(col("name"), " ").getItem(0))
pop_name = tempDf.groupBy("FirstName").count().orderBy(col("count").desc()).first()[0]
print(f"Najczestsze imie: {pop_name}")

curr_year = datetime.now().year
namesDf = namesDf.withColumn("Age", curr_year - year(col("date_of_birth")))

namesDf = namesDf.drop("bio", "death_details")

for col_name in namesDf.columns:
        new_col_name = col_name.replace("_", " ").title().replace(" ", "")
        namesDf = namesDf.withColumnRenamed(col_name, new_col_name)

namesDf = namesDf.orderBy(col("Name"))

display(namesDf.limit(10))

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
filePath = "dbfs:/FileStore/tables/Files/movies.csv"
namesDf = spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath)

display(moviesDf.limit(20))
moviesDf.explain()
moviesDf.columns
moviesDf = moviesDf.withColumn("CurrTime", lit(unix_timestamp(lit(datetime.now()))))

curr_year = datetime.now().year
moviesDf = moviesDf.withColumn("SinceRelease", curr_year - col("year"))

moviesDf = moviesDf.withColumn("BudgetNum", split(col("budget"), " ").getItem(1).cast("int"))

moviesDf = moviesDf.dropna()
display(moviesDf.limit(10))

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
namesDf = spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath)

display(ratingsDf.limit(10))
ratingsDf.explain()
ratingsDf.columns
ratingsDf = ratingsDf.withColumn("CurrTime", lit(unix_timestamp(lit(datetime.now()))))

ratingsDf_filtered = ratingsDf.filter(col("males_allages_avg_vote").isNotNull() & col("females_allages_avg_vote").isNotNull())

avg_male_rating = ratingsDf_filtered.select(avg(col("males_allages_avg_vote"))).collect()[0][0]
avg_female_rating = ratingsDf_filtered.select(avg(col("females_allages_avg_vote"))).collect()[0][0]

print(avg_female_rating)
print(avg_male_rating)


ratingsDf = ratingsDf.withColumn("total_votes_l", col("total_votes").cast("long"))

display(ratingsDf.limit(10))

#zadanie 3
ratingsDf_grouped = ratingsDf.groupBy("imdb_title_id").agg(avg("weighted_average_vote"))


print("Before groupby:")
ratingsDf.explain()
#hashed appeared
print("After groupby:")
ratingsDf_grouped.explain()